<a href="https://colab.research.google.com/github/hayakzan/MLP-notations/blob/main/MLP_notations_SYNTHESIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Synthesize the results into MIDI and .wav

In [57]:
!pip install mido
!pip install midi2audio
!apt install fluidsynth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
fluidsynth is already the newest version (2.1.1-2).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.


In [95]:
# -*- coding: utf-8 -*-

import json
from google.colab import drive
import shutil
import mido
from IPython.display import Audio
import os



In [59]:
# mount Google Drive
drive.mount('/content/drive')

source_file_path = '/content/drive/MyDrive/pitch_rhythm.json'

destination_directory = '/content'

shutil.copy(source_file_path, destination_directory)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/pitch_rhythm.json'

#### Process the JSON file

In [60]:
with open('pitch_rhythm.json') as json_file:
    data = json.load(json_file)

notation_excerpt = []

for group in data:
    input_pairs = []
    for pair in group:
        input_pairs.append(pair)

    notation_excerpt.append(input_pairs)

notation_excerpt[2]


[[10.5, 0.8114744066180365],
 [6.5, 1.209942887117903],
 [5.5, 1.5153243118303101],
 [7.5, 1.8658949119339887]]

In [61]:
# dictionary for notational position -> MIDI
pitch_dict = {0.0:52, 0.5:53, 1.0:55, 1.5:57, 2.0:59, 2.5:60, 3.0:62, 3.5:64, 4.0:65, 4.5:67, 5.0:69, 5.5:71, 6.0:72, 6.5:74, 7.0:76, 7.5:77, 8.0:79, 8.5:81, 9.0:83, 9.5:84, 10.0:86, 10.5:88, 11.0:89, 11.5:91, 12.0:93}


In [62]:
MIDI_pitches = []
MIDI_rhythms = []

for group in notation_excerpt:
    group_pitch_data = []
    for pairs in group:
        group_pitch_data.append(pairs[0])
    MIDI_pitches.append([pitch_dict[i] for i in group_pitch_data])


for group in notation_excerpt:
    group_rhythm_data = []
    for pairs in group:
        group_rhythm_data.append(pairs[1])
    MIDI_rhythms.append([int(240 * (rhythm / 1.235)) for rhythm in group_rhythm_data])

print("excerpt: ", notation_excerpt)
print("MIDI pitches: ", MIDI_pitches)
print("MIDI rhythms: ", MIDI_rhythms)

excerpt:  [[[5.0, 0.5155679697637757], [5.0, 1.3831723844516581], [9.0, 1.4937984907928312], [4.0, 2.333530484537259]], [[10.5, 0.5671113848377946], [4.5, 1.1255291031063108], [11.5, 1.7841389626124289], [5.5, 2.0711098003315347]], [[10.5, 0.8114744066180365], [6.5, 1.209942887117903], [5.5, 1.5153243118303101], [7.5, 1.8658949119339887]], [[5.5, 0.6288499678958566], [7.5, 1.0555497426869263], [8.5, 1.2097671599350592], [8.5, 1.5467349098527956]], [[5.5, 0.6737945023849068], [8.5, 1.0160596083312252], [4.5, 1.4645504503575524], [7.5, 1.6611128715631422]], [[9.5, 0.7220984972703434], [6.5, 0.8780545726883914], [8.5, 1.0851138366358726], [11.5, 1.3125289084440914]], [[8.5, 0.7027853392773753], [7.5, 0.8658932202351757], [5.5, 1.119364008249796], [11.5, 1.3579522344962023]], [[7.0, 0.9220838319125271], [6.0, 1.2845494592370534], [9.0, 1.9524483207124814], [11.0, 2.7035772006197547]], [[3.5, 0.6810584881776776], [11.5, 0.9572407431382669], [4.5, 1.359527561265557], [9.5, 1.542665461406131]

In [93]:
# Set the tempo and time signature values
tempo = mido.bpm2tempo(30)

# Set the ticks_per_beat value
ticks_per_beat = 960*2

# Calculate the duration of a quarter note in ticks
quarter_note_ticks = mido.second2tick(1.0 / 4, ticks_per_beat, tempo)

print(f"Duration of a quarter note in ticks: {quarter_note_ticks}")

Duration of a quarter note in ticks: 240.0
9
4
[100, 268, 290, 453]
100


#### Create MIDI files

In [94]:
# set the tempo
tempo = mido.bpm2tempo(30) 

for i in range(len(MIDI_rhythms)):
    # create a new MIDI file
    new_midi_file = mido.MidiFile()
    new_midi_file.ticks_per_beat = 960
    # create a new MIDI track
    new_track = mido.MidiTrack()
    new_track.append(mido.MetaMessage('set_tempo', tempo=tempo))  # Move tempo message here

    for j in range(len(MIDI_rhythms[i])):
        # add notes
        new_track.append(mido.Message('note_on', note=MIDI_pitches[i][j], velocity=64, time=MIDI_rhythms[i][j]))
        new_track.append(mido.Message('note_off', note=MIDI_pitches[i][j], velocity=64, time=MIDI_rhythms[i][j]))
    
    # add the track
    new_midi_file.tracks.append(new_track)
    
    # save the MIDI file
    filename = f'MLP-midi_{i+1}.mid'  # Use 'i+1' instead of 'track_num'
    new_midi_file.save(filename)


#### Convert the MIDI files into a .wav files (default SR: 44.1kHz)

In [100]:
len(MIDI_rhythms)
range(len(MIDI_rhythms)+1)

range(0, 10)

In [ ]:
# convert to .wav
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
!fluidsynth -ni font.sf2 MLP-midi.mid -F MLP-wav.wav -r 44100

Audio('MLP-wav.wav')

In [101]:
for i in range(len(MIDI_rhythms)+1):
    # Copy the soundfont file
    !cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
    
    # Convert MIDI to .wav
    midi_file = f'MLP-midi_{i}.mid'
    wav_file = f'MLP-wav_{i}.wav'
    !fluidsynth -ni font.sf2 {midi_file} -F {wav_file} -r 44100
    
    # Check if the file was created successfully
    if os.path.isfile(wav_file):
        print(f"Conversion successful: {midi_file} -> {wav_file}")
    else:
        print(f"Conversion failed: {midi_file}")



FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Parameter 'MLP-midi_0.mid' not a SoundFont or MIDI file or error occurred identifying it.
No midi file specified!
Conversion failed: MLP-midi_0.mid
FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file 'MLP-wav_1.wav'..
Conversion successful: MLP-midi_1.mid -> MLP-wav_1.wav
FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file 'MLP-wav_2.wav'..
Conversion successful: MLP-midi_2.mid -> MLP-wav_2.wav
FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL licen